In [1]:
MODEL_FOLDER = "../models/"
DATA_FOLDER = "../data/samples/" 
RESULT_FOLDER = "../results/" 

SAMPLES_G = ["SGMH"] #["SGSH", "SGSL", "SGMH", "SGML"]
SAMPLES_P = ["SPSH", "SPSL"]
SAMPLES_S = ["SGMH"] #, "SCSL"]

In [2]:
MODELS = {
    "RETRIEVER":  (0, "SPhilBERTa"),
    "RERANKER_P": (0, "LaBERTa"),
    "RERANKER_S": (1, "CES-5E")
}

PIPELINE_CONFIGS = [
    {"threshold": t} 
    for t in [0.5, 0.6, 0.7]
]
RETRIEVAL_CONFIGS = [
    {"k": k, "threshold": t}
    for k in [2, 5, 8]
    for t in [0.5, 0.6, 0.7, 0.8]
]

import benchmarking
benchmarking.BenchmarkRunner(
        MODEL_FOLDER, [name if select else None for select, name in MODELS.values()],
        DATA_FOLDER, {'G': SAMPLES_G, 'F': SAMPLES_F, 'P': SAMPLES_P, 'C': SAMPLES_C},
        RESULT_FOLDER,
        PIPELINE_CONFIGS, RETRIEVAL_CONFIGS, RERANKER_CONFIGS
    ).benchmark()

Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:14<00:00,  5.13it/s]

Reranker
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       799
           1       0.97      0.99      0.98       201

    accuracy                           0.99      1000
   macro avg       0.98      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

